# Instalacao das dependencias e Importacao das bibliotecas 

In [ ]:
pip install gymnasium


In [ ]:
! pip install "stable-baselines3[extra]"
! pip install -q swig
! pip install -q "gymnasium[box2d]"


In [ ]:
import gymnasium as gym
from collections import defaultdict

from stable_baselines3 import PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

import os
import warnings
import numpy as np
import random

## Treinamento do modelo

Esses parâmetros são configurados para balancear o desempenho e a estabilidade do treinamento do agente PPO no ambiente CarRacing-v2. Ajustar esses hiperparâmetros pode ser necessário para otimizar o desempenho do agente para tarefas específicas e pode requerer experimentação baseada nos resultados de treinamento e nos gráficos de desempenho observados através do TensorBoard.

In [ ]:
model = PPO("CnnPolicy", 'CarRacing-v2',#Arquitetura da rede neural e ambiente de treinamento
            tensorboard_log = 'training/logs', #Diretório onde os logs do TensorBoard serão armazenados.
            batch_size = 512, #Tamanho do lote de amostras usadas em cada atualização de gradiente. Um tamanho de lote maior pode levar a uma estimativa mais estável do gradiente, mas também requer mais memória e pode tornar o treinamento mais lento.
            clip_range = 0.2, #valor de clipe para a probabilidade de razão.
            ent_coef = 0.0, #Coeficiente de entropia. A entropia é usada para incentivar a exploração adicionando uma penalidade por políticas determinísticas. Um valor de 0.0 significa que não há incentivo adicional para a exploração.
            gae_lambda = 0.95,#O fator lambda para o Generalized Advantage Estimation (GAE).
            gamma = 0.99, #Fator de desconto. Um valor de 0.99 significa que o agente valoriza mais as recompensas futuras próximas. É um fator comum em muitos problemas de aprendizado por reforço.
            learning_rate = 0.0003,#Taxa de aprendizado do otimizador.Define a velocidade com que o modelo se ajusta aos gradientes. Um valor de 0.0003 é um ponto de partida comum, proporcionando um bom equilíbrio entre a convergência rápida e a estabilidade do treinamento.
            max_grad_norm = 0.5,#Norma máxima para os gradientes. Clipping dos gradientes para evitar explosões de gradiente, que podem desestabilizar o treinamento. Um valor de 0.5 é frequentemente utilizado para manter os gradientes sob controle.
            n_epochs = 10)#Número de épocas para cada atualização de política. Define quantas vezes o modelo passa pelos dados de treinamento em cada atualização. Mais épocas podem levar a um melhor ajuste, mas também aumentam o tempo de treinamento

In [ ]:
model.learn(total_timesteps = 1000000, log_interval = 5, progress_bar = True)


In [ ]:
model.save('training/trained_models3/PPO_400kCarRacing_whp')


In [ ]:
env = gym.make('CarRacing-v2', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

## Criacao dos videos para interpretar os modelos

In [ ]:
!pip install opencv-python

In [ ]:
# Import the OpenCV library
import cv2 

In [ ]:
# Importa o ambiente CarRacing-v2 do Gym e inicializa com o modo de renderização rgb_array
env = gym.make('CarRacing-v2', render_mode='rgb_array')
# Cria um DummyVecEnv para compatibilidade com o stable_baselines3
env = DummyVecEnv([lambda: env])

# Define parâmetros do vídeo
width, height = 600, 400  # Largura e altura do vídeo
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec para compressão do vídeo
video_writer = cv2.VideoWriter("Car Racing3.mp4", fourcc, 30.0, (width, height))  # Inicializa o gravador de vídeo

# Número de episódios a serem gravados
episodes = 10

# Loop sobre os episódios
for episode in range(1, episodes + 1):
    # Reinicia o ambiente para o início de um novo episódio
    observation = env.reset()
    done, score = False, 0  # Inicializa variáveis de controle

    # Loop até que o episódio termine
    while not done:
        # Prediz a ação usando o modelo treinado
        action, _ = model.predict(observation)
        step = env.step(action)  # Executa a ação no ambiente

        # Extrai informações do passo retornado pelo ambiente
        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward  # Acumula a recompensa

        # Grava o ambiente
        frame = env.render()  # Renderiza o ambiente e obtém o frame
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Converte o frame de RGB para BGR
        frame = cv2.resize(frame, (width, height))  # Redimensiona o frame para as dimensões desejadas

        # Escreve o frame no vídeo
        video_writer.write(frame)

    # Exibe a pontuação do episódio
    print(f"Episode {episode} score: {score}")

# Libera o gravador de vídeo
video_writer.release()
# Fecha o ambiente
env.close()
# Destroi todas as janelas do OpenCV
cv2.destroyAllWindows()

## Ler o modelo ja treinado ou buscar da pasta - para salvar um novo treinamento alterar o nome da pasta trained_models

In [ ]:
model = PPO.load('training/trained_models3/PPO_400kCarRacing_whp')

In [1]:
%reload_ext tensorboard
%tensorboard --logdir training/logs

In [ ]:
env = gym.make('CarRacing-v2', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

# Ler os videos das pastas salvas:

In [ ]:
!pip install opencv-python
!pip install opencv-python-headless
!pip install numpy


Leitura do video - trocar os nomes

In [ ]:
import cv2
from IPython.display import display, clear_output, Image

def display_video(video_path):
    # Abre o vídeo
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        # Lê um quadro do vídeo
        ret, frame = cap.read()
        if not ret:
            break
        
        # Converte a imagem de BGR (usado pelo OpenCV) para RGB (usado pelo Jupyter)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Converte o quadro para imagem no formato que o Jupyter aceita
        _, img = cv2.imencode('.jpg', frame)
        display(Image(data=img.tobytes()))
        
        # Limpa a saída para o próximo quadro
        clear_output(wait=True)
    
    cap.release()

# Chama a função passando o caminho do vídeo
display_video('Car Racing3.mp4')


Resolver o problema do display trazendo como widget do Jupyter